# tensorcircuit SDK for QCloud（230203 ver）

## import the package

``apis`` is temporarily as the entry point submodule for qcloud

In [ ]:
import tensorcircuit as tc
from tensorcircuit.cloud import apis
from tensorcircuit.cloud.wrapper import batch_expectation_ps
from tensorcircuit.compiler.qiskit_compiler import qiskit_compile
import numpy as np

## setup the token

The users need an API token from tQuK to connect to the server and submit tasks, the token only need be set once and it is then written to the local computer

In [ ]:
apis.set_token("foobar")
# only required running once for a given laptop

## list providers/devices/properties

Get basic info of devices and device information

In [ ]:
apis.list_providers()

In [ ]:
apis.list_devices("tencent")

In [ ]:
apis.list_properties(device="9gmon")

In [ ]:
d = apis.get_device("9gmon")

In [ ]:
d.list_properties()["bits"][8]

In [ ]:
d.topology()

In [ ]:
d.native_gates()

In [ ]:
d.topology_graph(visualize=True)

## Task submit and the results

Basic task submission syntax below, here we use a simulator backend on tQuK `simulator:tc`

In [ ]:
c = tc.Circuit(1)
c.H(0)

t = apis.submit_task(device="simulator:tc", circuit=c, shots=1024)
print(t.details())
t.results(blocked=True)

``blocked=True`` can wait until the task is finished or failed (rasing an error)

In [ ]:
t.status()

In [ ]:
t.get_device()

resubmit a job with the same source (device/shots) and command (circuit)


In [ ]:
t1 = t.resubmit()
t1.details(blocked=True, prettify=True)

``t.details`` can also permit the ``blocked=True`` option, which waits until the task is finished or failed (no error raised).

Also note by using ``prettfiy=True`` option, we have python datatime object for the timestamp which is easy to read but hard for io (not directly json serializable anymore) 

## local provider enable quick debugging and testing

TC comes with a local provider which behaves as a simple cloud provider but run the circuit locally

In [ ]:
apis.set_provider("local")
# using tc simulator on local device: your own laptop is your server
apis.list_devices()

In [ ]:
c = tc.Circuit(2)
c.h(0)
c.cx(0, 1)

# exactly the same API as tQuK
t = apis.submit_task(circuit=c, device="testing", shots=8192)
t.results(blocked=True)

In [ ]:
tl = apis.list_tasks()
tl

In [ ]:
id_ = tl[0].__str__()
print(id_)
t = apis.get_task(id_)
t.details()

In [ ]:
id_ = tl[0].__str__()
print(id_.split("~~")[1])
t = apis.get_task(id_)
t.details()

As shown above, the task can be indexed either with device information or not (as long as we use ``set_provider``)

In [ ]:
# back to tencent server for demonstration below
apis.set_provider("tencent")

## GHZ state on real device and readout mitigation

In [ ]:
nqubit = 9
shots = 4096
c = tc.Circuit(nqubit)
c.H(8)
c.cnot(8, 4)
c.cnot(4, 0)
c.cnot(0, 2)
c.cnot(2, 6)

# above we dirct assign physical qubits

t = apis.submit_task(
    circuit=c, shots=shots, device="9gmon", enable_qos_qubit_mapping=False
)
raw_count = t.results(blocked=True)
# blocked = True will block the process until the result is returned
# the default behavior is blocked=False, where only one query is made and raise error when the task is incomplete

# note we explicitly turn off qubit mapping from qos, which gurantee our logical circuit are identical to the physical one.
# but one should ensure the topology link in the logical circuit is compatible with the target device

In the below, we use tensorcircuit builtin powerful tool for readout mitigation: ``tc.results.rem.ReadoutMit``, it supports various method for calibriation and mitigation

In [ ]:
mit = tc.results.rem.ReadoutMit("9gmon?o=0")
# here o=0 is a short for disable qubit mapping and gate decomposition at the backend server
mit.cals_from_system(nqubit, shots, method="local")
# local calibriation
miti_count = mit.apply_correction(raw_count, nqubit, method="constrained_least_square")

By attaching ``?o=0`` after the device string, we have the same effect of setting ``enable_qos_qubit_mapping=False`` (o=1)
and ``enable_qos_gate_decomposition=False`` (o=2), and both of them are by default True (o=3).

We can define the REM class by using more customizable function.

In [ ]:
def run(cs, shots):
    """batch mode"""
    ts = apis.submit_task(
        circuit=cs, shots=shots, device="9gmon", enable_qos_qubit_mapping=False
    )
    return [t.results(blocked=True) for t in ts]


mit = tc.results.rem.ReadoutMit(run)
mit.cals_from_system(nqubit, shots, method="local")

In [ ]:
raw_count = tc.results.counts.marginal_count(raw_count, [8, 4, 0, 2, 6])
miti_count = tc.results.counts.marginal_count(miti_count, [8, 4, 0, 2, 6])
# only keep the result for qubit 8, 4, 0, 2, 6 and in that exact order

In [ ]:
tc.results.counts.plot_histogram([raw_count, miti_count])

In [ ]:
ideal_count = tc.results.counts.vec2count(c.probability(), prune=True)
# we can obtain analytic count results by ``c.probability()`` method, and ``vec2count`` with transform the vector as a dict

ideal_count = tc.results.counts.marginal_count(ideal_count, [8, 4, 0, 2, 6])
tc.results.counts.kl_divergence(
    ideal_count, raw_count
), tc.results.counts.kl_divergence(ideal_count, miti_count)

In [ ]:
# we can directly check local readout matrix on each qubit
print("readout matrix")
for i, m in enumerate(mit.single_qubit_cals):
    print("qubit %s:" % i)
    print(m)

Apart from calibriation from real experiments, we can access the readout error matrix from API (which is fast but may be not that up to date)

In [ ]:
mit = tc.results.rem.ReadoutMit("9gmon?o=0")
mit.cals_from_api(nqubit)
mit.single_qubit_cals[0]

## Abstraction of three layers of qubits and the mappings

In the above example, the circuit is not compiled by the frontend: tc or backend: qos, in the follows, we will introduce circuit compiling and the new abstraction on different level of qubits.

New abstraction on qubits: positional qubits, logical qubits, physical qubits, we need two more mappings: ``positional_logical_mapping`` and ``logical_physical_mapping``.

The separation between positional and logical qubits is due to partial measurement, while the seperation between logical and physical qubits are from circuit compiling onto hardware, including swap inserting (where the last swap is omitted, current qos behavior), qubit routing (i.e. initial mapping).

Now we do the GHZ preparation on another chip, but use mapping and partial measurement abstraction this time

In [ ]:
# logical circuit for GHZ-5

c = tc.Circuit(5)
c.h(0)
for i in range(4):
    c.cx(i, i + 1)
for i in range(5):
    c.measure_instruction(i)

# We map the circuit on the physical qubits by hand

c1 = c.initial_mapping({0: 8, 1: 4, 2: 0, 3: 2, 4: 6}, n=9)
positional_logical_mapping = c1.get_positional_logical_mapping()
positional_logical_mapping

In [ ]:
c1.draw()  # circuit after mapping

In [ ]:
t = apis.submit_task(
    circuit=c1, shots=shots, device="9gmon", enable_qos_qubit_mapping=False
)
raw_count = t.results(blocked=True)

In [ ]:
logical_physical_mapping = t.details()["optimization"]["pairs"]
logical_physical_mapping
# this mapping is identical since we disable qos qubit mapping above

In [ ]:
mit = tc.results.rem.ReadoutMit("9gmon?o=0")
mit.cals_from_system(9, shots, method="local")
miti_count = mit.apply_correction(
    raw_count,
    [8, 4, 0, 2, 6],
    positional_logical_mapping=positional_logical_mapping,
    logical_physical_mapping=logical_physical_mapping,
    method="square",
)

In [ ]:
plot_histogram([raw_count, miti_count])

We can have another way to understand logical qubits: we could treat 0-4 in the original circuit as logical qubits, then we will have the following convention and the circuit after initial mapping as the physical one (abstraction reference shift)

In [ ]:
miti_count = mit.apply_correction(
    raw_count,
    [0, 1, 2, 3, 4],
    positional_logical_mapping=None,
    logical_physical_mapping={0: 8, 1: 4, 2: 0, 3: 2, 4: 6},
    method="square",
)
# note how the None by default implies an identity mapping

In [ ]:
plot_histogram([raw_count, miti_count])
# the results should be exactly the same, since they are just the same thing using different reference system

The above abstraction is rather low level where the compiling is done by hand and we recommend the following api for users (**the highly recommended way**).

The recommended approach heavily depends on the frontend compiling via qiskit (builtin support in tc).

In [ ]:
# 0. acquire readout mitigation class

mit = tc.results.rem.ReadoutMit("20xmon?o=0")
mit.cals_from_system(20)

# 1. define the logical circuit

n = 5
c = tc.Circuit(n)
c.h(0)
for i in range(n - 1):
    c.cx(i, i + 1)
for i in reversed(range(n)):
    c.measure_instruction(i)

# 2. compile the circuit

d = apis.get_device("20xmon")

c1, info = qiskit_compile(
    c,
    compiled_options={
        "basis_gates": d.native_gates(),
        "optimization_level": 3,
        "coupling_map": d.topology(),
    },
)


# 3. submit the job and get the raw result

t = apis.submit_task(
    circuit=c1,
    shots=8192,
    device=d,
    enable_qos_qubit_mapping=False,
    enable_qos_gate_decomposition=False,
)
raw_count = t.results(blocked=True)

# 4. obtain the mitigated result in terms of distribution or expectation

print("distribution", mit.apply_correction(raw_count, n, method="square", **info))
print("<Z0Z1>", mit.expectation(raw_count, [0, 1], **info))

In [ ]:
info  # compiling info and the qubit mapping are recorded automatically

In [ ]:
tc.results.counts.plot_histogram(
    [raw_count, mit.apply_correction(raw_count, n, method="square", **info)]
)

And the **all-in-one API**: ``batch_expectation_ps`` with circuit generating, grouping, compiling, optimization and error mitigation support is as shown below, the API is also consistent with numerical simulations, basically the API capture all the workflow shown in above cell with extra enhancement

In [ ]:
c = tc.Circuit(2)
c.h(0)
c.cz(0, 1)
c.x(1)
print("numerical results: [<X_0>, <X_0Z_1>]", batch_expectation_ps(c, [[1, 0], [1, 3]]))
print(
    "hardware results: [<X_0>, <X_0Z_1>]",
    batch_expectation_ps(c, [[1, 0], [1, 3]], "20xmon"),
)
print(
    "numerical results: <X_0> + 0.5* <X_0Z_1>",
    batch_expectation_ps(c, [[1, 0], [1, 3]], ws=[1, 0.5]),
)
print(
    "hardware results: <X_0> + 0.5* <X_0Z_1>",
    batch_expectation_ps(c, [[1, 0], [1, 3]], "20xmon", ws=[1, 0.5]),
)

batch submission is possible with multiple circuits in a list and the return is a list of task, respectively. The batch mechanism are supported both on real chips and simulators.

In [ ]:
# we can also do a batch submission for the real hardware chip, simply by provide a circuit list

c = tc.Circuit(2)
c.h(0)

c1 = tc.Circuit(2)
c1.h(1)

ts = apis.submit_task(device="20xmon", circuit=[c, c1], shots=1024)

for t in ts:
    print(t.results(blocked=True))

## measure on partial of the qubits

Note the return order should ideally follow the measure order in the instructions

In [ ]:
# directly partial measure via qiskit

from qiskit.circuit import QuantumCircuit

qc = QuantumCircuit(9, 9)
qc.x(8)
qc.x(6)
qc.measure(8, 8)
qc.measure(2, 2)
qc.measure(6, 6)

t = apis.submit_task(circuit=qc, shots=shots, device="9gmon?o=0")
print(t.results(blocked=True))

 The above case also indicates that tc ``submit_task`` API directly support Qiskit ``QuantumCircuit`` object

In [ ]:
# directly partial measure on tc

# recommended approach

nqubit = 9
shots = 4096
c = tc.Circuit(nqubit)
c.x(8)
c.x(6)
c.measure_instruction(8)
c.measure_instruction(2)
c.measure_instruction(6)

t = apis.submit_task(circuit=c, shots=shots, device="9gmon?o=0")
print(t.results(blocked=True))
print(c.get_positional_logical_mapping())

partial measurment also supported via the simulator on the cloud

In [ ]:
nqubit = 9
shots = 4096
c = tc.Circuit(nqubit)
c.x(8)
c.x(6)
c.measure_instruction(8)
c.measure_instruction(2)
c.measure_instruction(6)

t = apis.submit_task(circuit=c, shots=shots, device="simulator:tc")
print(t.results(blocked=True))

In [ ]:
nqubit = 9
shots = 4096
c = tc.Circuit(nqubit)
c.x(8)
c.x(6)
c.measure_instruction(8)
c.measure_instruction(2)
c.measure_instruction(6)

t = apis.submit_task(circuit=c, shots=shots, device="simulator:aer")
print(t.results(blocked=True))

## two level compiling system

We provide compiling support at frond end (via tc-qiskit pipeline) and at back end (in qos).
The front end option is enabled by ``compiled=True`` (default to False) and also with an optional dict for ``qiskit.transpile`` arguments called ``compiled_options``. For advanced users, we recommand you to separately deal with the circuit compiling and submission as we discussed above as the recommended approach. The backend qos compiling is controlled by ``enable_qos_qubit_mapping`` and ``enable_qos_gate_decomposition`` (all default to True). The ``?o=int`` str after the device name can overide qos compiling options. We strongly recommend the users only use one part of the compiling in case confusing and conflicts. For front end compiling, though the built-in compiling via ``compiled`` switch in ``submit_task`` is handy, we recommend the advanced user to use standalone compiling module as shown above, i.e. explicitly call ``qiskit_compile``, the advantage for the latter is we can obtain qubit mapping information at the same time for further error mitigation pipelines.

In [ ]:
# directly use built-in mitigation with expectation evaluation + front-end (tc/qiskit) compiling system

nqubit = 3
shots = 8192
c = tc.Circuit(nqubit)
c.h(0)
c.rz(0, theta=0.4)
c.x(0)
c.y(0)
c.h(1)
c.rx(2, theta=0.7)
c.ry(1, theta=-1.2)
c.cnot(0, 1)
c.cnot(2, 0)
c.h(1)
c.x(2)

print("exact: ", [np.real(c.expectation_ps(z=[i])).tolist() for i in range(nqubit)])

t = apis.submit_task(
    circuit=c,
    shots=shots,
    device="9gmon",
    compiled=True,
    enable_qos_qubit_mapping=False,
    enable_qos_gate_decomposition=False,
)

ct = t.results(blocked=True)

mit = tc.results.readout_mitigation.ReadoutMit("9gmon?o=0")
mit.cals_from_system(3, shots=8192, method="local")

print(
    "experiments (mitigated directly via expectation): ",
    [mit.expectation(ct, [i]) for i in range(nqubit)],
)

# no need to provider mapping in mit as there is no mapping in this case,
# compiled=True itself doesn't enable front end qubit routing

print(
    "experiments (mitigated using lstm): ",
    [
        tc.results.counts.expectation(mit.apply_correction(ct, 3, method="square"), [i])
        for i in range(nqubit)
    ],
)

In [ ]:
c.draw()  # target circuit: mimic a VQA case

In [ ]:
# use backend compiling system enabled by qos and the very handy built-in auto mitigation
# (only works without qubit mapping at front end)

nqubit = 3
shots = 8192
c = tc.Circuit(nqubit)
c.h(0)
c.rz(0, theta=0.4)
c.x(0)
c.y(0)
c.h(1)
c.rx(2, theta=0.7)
c.ry(1, theta=-1.2)
c.cnot(0, 1)
c.cnot(2, 0)
c.h(1)
c.x(2)

print("exact: ", [np.real(c.expectation_ps(z=[i])) for i in range(nqubit)])

t = apis.submit_task(
    circuit=c,
    shots=shots,
    device="9gmon",
    compiled=False,
    enable_qos_qubit_mapping=True,
    enable_qos_gate_decomposition=True,
)

ct = t.results(blocked=True, mitigated=True)
# auto mitigation with backend qubit mapping considered


print(
    "experiments (mitigated): ",
    [tc.results.counts.expectation(ct, [i]) for i in range(nqubit)],
)

In [ ]:
# inspect compiling results from the tc and qos for the task, we can directly get the circuit objects from prettified details

c_complied_before_qos = t.details(prettify=True)["frontend"]
c_complied_after_qos = t.details(prettify=True)["backend"]

In [ ]:
c_complied_before_qos.draw()

In [ ]:
c_complied_after_qos.draw(output="mpl")

dry run mode to query compiled circuit only from qos (not really sending the circuit to chips), we can use ``qos_dry_run=True`` option


In [ ]:
nqubit = 3
shots = 8192
c = tc.Circuit(nqubit)
c.h(0)
c.h(1)
c.rx(2, theta=0.7)
c.ry(1, theta=-1.2)
c.cnot(0, 1)
c.cnot(2, 0)
c.h(1)


t = apis.submit_task(
    circuit=c,
    shots=shots,
    device="9gmon",
    compiled=True,
    enable_qos_qubit_mapping=True,
    enable_qos_gate_decomposition=True,
    qos_dry_run=True,
)

In [ ]:
t.details(prettify=True, blocked=True)["backend"].draw()

## scalable readout simulation and mitigation

Via TensorCircuit, we provide the capability to do scalable (20+ qubits) readout error simulation and mitigation

In [ ]:
# scalable readout error simulation on tQuK with tensorcircuit backend using tensor network approach

c = tc.Circuit(3)
t = apis.submit_task(circuit=c, device="simulator:tcn1", shots=8192)
t.results(blocked=True)

In [ ]:
t.results(mitigated=True)

In [ ]:
c = tc.Circuit(25)
t = apis.submit_task(circuit=c, device="simulator:tcn1", shots=8192)
t.results(blocked=True)

Simulator device also support batch submission

In [ ]:
# batch submission to the simulator
cs = []
for i in range(15):
    c = tc.Circuit(15)
    c.x(i)
    cs.append(c)
ts = apis.submit_task(circuit=cs, device="simulator:tcn1", shots=8192)

In [ ]:
# mitigated with m3 scalable on count dict
c = tc.Circuit(15)
c.x(0)
t = apis.submit_task(circuit=c, device="simulator:tcn1", shots=8192)

mit = tc.results.readout_mitigation.ReadoutMit("simulator:tcn1")
mit.cals_from_system(15)

raw_count = t.results(blocked=True)
mit.apply_correction(raw_count, 15, method="M3_auto")

In [ ]:
# mitigated scalable directly on expectation: not a wrapper for count but a new algorithm!
# see eq 6 in https://arxiv.org/pdf/2006.14044.pdf

mit.expectation(raw_count, [0])

## list task and get previous task

get history tasks and their details, so that your experimental data are always accessible with detailed meta data on the cloud

In [ ]:
apis.list_tasks()

In [ ]:
apis.list_tasks(device="9gmon")

In [ ]:
t = apis.get_task("d77bec2f-ab07-4dbc-a273-caa8b23a921c")

In [ ]:
t.details()

In [ ]:
t = apis.get_task("tencent::9gmon~~e32bb488-5ee9-4b07-8217-1e78ceb4bde3")

In [ ]:
t.details(prettify=True)

In [ ]:
t.results()